# Tensorflow-图像滤波

In [5]:
%matplotlib inline
from matplotlib import gridspec as gridspec
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np

In [8]:
a = np.zeros([3, 3, 3, 3])
a[1, 1, :, :] = 0.25
a[0, 1, :, :] = 0.125
a[1, 0, :, :] = 0.125
a[2, 1, :, :] = 0.125
a[1, 2, :, :] = 0.125
a[0, 0, :, :] = 0.0625
a[0, 2, :, :] = 0.0625
a[2, 0, :, :] = 0.0625
a[2, 2, :, :] = 0.0625
BLUR_FILTER_RGB = tf.constant(a.copy(), dtype=tf.float32)

a = np.zeros([3, 3, 1, 1])
# a[1, 1, :, :] = 0.25
# a[0, 1, :, :] = 0.125
# a[1, 0, :, :] = 0.125
# a[2, 1, :, :] = 0.125
# a[1, 2, :, :] = 0.125
# a[0, 0, :, :] = 0.0625
# a[0, 2, :, :] = 0.0625
# a[2, 0, :, :] = 0.0625
# a[2, 2, :, :] = 0.0625
a[1, 1, :, :] = 1.0
a[0, 1, :, :] = 1.0
a[1, 0, :, :] = 1.0
a[2, 1, :, :] = 1.0
a[1, 2, :, :] = 1.0
a[0, 0, :, :] = 1.0
a[0, 2, :, :] = 1.0
a[2, 0, :, :] = 1.0
a[2, 2, :, :] = 1.0
BLUR_FILTER = tf.constant(a.copy(), dtype=tf.float32)

a = np.zeros([3, 3, 3, 3])
a[1, 1, :, :] = 5
a[0, 1, :, :] = -1
a[1, 0, :, :] = -1
a[2, 1, :, :] = -1
a[1, 2, :, :] = -1

SHARPEN_FILTER_RGB = tf.constant(a.copy(), dtype=tf.float32)

a = np.zeros([3, 3, 1, 1])
a[1, 1, :, :] = 5
a[0, 1, :, :] = -1
a[1, 0, :, :] = -1
a[2, 1, :, :] = -1
a[1, 2, :, :] = -1

SHARPEN_FILTER = tf.constant(a.copy(), dtype=tf.float32)

# a = np.zeros([3, 3, 3, 3])
# a[:, :, :, :] = -1
# a[1, 1, :, :] = 8

# EDGE_FILTER_RGB = tf.constant(a, dtype=tf.float32)

EDGE_FILTER_RGB = tf.constant([
        [[[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
        [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
        [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]]],
        [[[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
        [[ 8., 0., 0.], [ 0., 8., 0.], [ 0., 0., 8.]],
        [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]]],
        [[[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
        [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
        [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]]]
])

a = np.zeros([3, 3, 1, 1])
# a[:, :, :, :] = -1
# a[1, 1, :, :] = 8
a[0, 1, :, :] = -1
a[1, 0, :, :] = -1
a[1, 2, :, :] = -1
a[2, 1, :, :] = -1
a[1, 1, :, :] = 4

EDGE_FILTER = tf.constant(a.copy(), dtype=tf.float32)

a = np.zeros([3, 3, 3, 3])
a[0, :, :, :] = 1
a[0, 1, :, :] = 2 # originally 2
a[2, :, :, :] = -1
a[2, 1, :, :] = -2

TOP_SOBEL_RGB = tf.constant(a.copy(), dtype=tf.float32)

a = np.zeros([3, 3, 1, 1])
a[0, :, :, :] = 1
a[0, 1, :, :] = 2 # originally 2
a[2, :, :, :] = -1
a[2, 1, :, :] = -2

TOP_SOBEL = tf.constant(a.copy(), dtype=tf.float32)

a = np.zeros([3, 3, 3, 3])
a[0, 0, :, :] = -2
a[0, 1, :, :] = -1 
a[1, 0, :, :] = -1
a[1, 1, :, :] = 1
a[1, 2, :, :] = 1
a[2, 1, :, :] = 1
a[2, 2, :, :] = 2

EMBOSS_FILTER_RGB = tf.constant(a.copy(), dtype=tf.float32)

a = np.zeros([3, 3, 1, 1])
a[0, 0, :, :] = -2
a[0, 1, :, :] = -1 
a[1, 0, :, :] = -1
a[1, 1, :, :] = 1
a[1, 2, :, :] = 1
a[2, 1, :, :] = 1
a[2, 2, :, :] = 2
EMBOSS_FILTER = tf.constant(a.copy(), dtype=tf.float32)

In [9]:
def read_one_image(filename):
    """ This is just to demonstrate how to open an image in TensorFlow,
    but it's actually a lot easier to use Pillow 
    """
    filename_queue = tf.train.string_input_producer([filename])
    image_reader = tf.WholeFileReader()
    _, image_file = image_reader.read(filename_queue)
    image = tf.image.decode_jpeg(image_file, channels=3)
    image = tf.cast(image, tf.float32) / 256.0 # cast to float to make conv2d work
    return image

In [10]:
def convolve(image, kernels, rgb=True, strides=[1, 3, 3, 1], padding='SAME'):
    images = [image[0]]
    for i, kernel in enumerate(kernels):
        filtered_image = tf.nn.conv2d(image, kernel, strides=strides, padding=padding)[0]
        if i == 2:
            filtered_image = tf.minimum(tf.nn.relu(filtered_image), 255)
        images.append(filtered_image)
    return images

In [11]:
def get_real_images(images):
    with tf.Session() as sess:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        images = sess.run(images)
        coord.request_stop()
        coord.join(threads)
    return images

In [12]:
def show_images(images, rgb=True):
    gs = gridspec.GridSpec(1, len(images))
    for i, image in enumerate(images):
        plt.subplot(gs[0, i])
        if rgb:
            plt.imshow(image)
        else: 
            image = image.reshape(image.shape[0], image.shape[1])
            plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.show()

In [13]:
FILENAME = '../datas/f2.jpg'

In [15]:
def main():
    rgb = True
    if rgb:
        kernels_list = [BLUR_FILTER_RGB, SHARPEN_FILTER_RGB, EDGE_FILTER_RGB, 
                    TOP_SOBEL_RGB, EMBOSS_FILTER_RGB]
    else:
        kernels_list = [BLUR_FILTER, SHARPEN_FILTER, EDGE_FILTER, 
                    TOP_SOBEL, EMBOSS_FILTER]

    image = read_one_image(FILENAME)
    if not rgb:
        image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, 0) # to make it into a batch of 1 element
    images = convolve(image, kernels_list, rgb)
    images = get_real_images(images)
    show_images(images, rgb)